<a href="https://colab.research.google.com/github/aldisulthony/Pemrograman-Python/blob/main/Aldi_Sulthony_Susilo_1101172222_Image_Classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Google Drive cloud database

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly&response_type=code

Enter your authorization code:
4/1AY0e-g6Et8T3WY9lq--yoZjYxUu730Z_sy8OQrDw2qJoFROQ602VJ0obY0U


Importing the folder that contains the dataset needed for image classification modeling

In [ ]:
cd /content/drive/MyDrive/data

List the folder to makesure that the dataset includes in the specified folder

In [ ]:
ls

In [ ]:
# Import necessary modules and packages
from keras.models import Sequential
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from PIL import Image
from imutils import paths
import numpy as np
import os

In [ ]:
# Create the image path with "paths" in the input data dataset directory
# List the images and its labels
print("[INFO loading images..")
imagePaths = paths.list_images("dataset")
data = []
labels = []

In [ ]:
# Loop over the input images using "for" function:
for imagePath in imagePaths:
  # Load the input image from the selected image path
  image = Image.open(imagePath)

  # Resize it to 64x64 pixels, the pixel intensities to the range [0,1] and then updateour image list
  # Conducting the prepocessing and normalize process to the image
  image = np.array(image.resize((64,64))) / 255.0 
  data.append(image)

  # Extract the class label from the path file and update the labels list
  label = imagePath.split(os.path.sep)[-2]
  labels.append(label)

In [ ]:
# Visualize the input image label list
print(labels)

In [ ]:
# Encode the input image labels, convert them from strings to integers to make easier for computer to proceed
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [ ]:
# Visualize the converted input label images
print(labels)

In [ ]:
# Perform a training and testing split data for modeling
# Using 80% of the data for training and 20% of the data for evaluation
(trainX, testX, trainY, testY) = train_test_split(np.array(data), np.array(labels), test_size=0.20, shuffle=True)

# Visualize train and test X shape
print(trainX.shape)
print(testX.shape)

In [ ]:
# Conducting Convolutional Neural Network and Fully Connected Feedforward Network

# Import necessary modules and packages
from keras.layers import Convolution2D, MaxPooling2D
from keras.models import Sequential
from keras.layers.core import Flatten, Dense

# Proceed the CNN stages (2 times of Convolution and Max Pooling)
model = Sequential()
model.add(Convolution2D(8, (3,3), activation="relu", input_shape=(64,64,3))) # the input shape is supposed to be the same as resize input image [64x64] for RGB image
model.add(MaxPool2D(2,2))
model.add(Convolution2D(16, (3,3), activation="relu"))
model.add(MAxPool2D(2,2))

# Conduct the Fully Connected Layer  stages
model.add(Flatten) # remap the weight to 1D
model.add(Dense(100, activation='relu'))
model.add(Dense(5, activation='softmax')) # 5 for attributes that is use for this modeling

# Perform the model summary
model.summary

In [ ]:
# Train the model using the Adam Optimizer
print('[INFO] training network...')
opt = Adam(lr=1e-3, decay=1e-3/50)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics["accuracy"])

In [ ]:
H = model.fit(trainX,trainY, validation_data=(testX, testY,), epochs=25, batch_size=32)

In [ ]:
# Visualixe the trained model

# Import Matplotlib
import matplotlib.pyplot as plt

print(H.history.keys())

# Summarize history for accuracy
plt.plot(H.history['accuracy'])
plt.plot(H.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Summarize history for loss
plt.plot(H.history['loss'])
plt.plot(H.history['val_loss'])
plt.title('Modell Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Evaluate the model by using F1 Score Table
print('[INFO] Evaluating Network...')
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=lb.classes_))

In [ ]:
# Save the Model
model.save('cnnmodel_leaf')

In [ ]:
# Perform image testing by using the saved moddel by using cv2

# Import necessary modules
import cv2
import matplotlib.pyplot as plt 
image_test = 'leaf_test.jpg'

In [ ]:
# Read and Show the image test file by using cv2 module
img_array = cv2.imread(image_test)

# Read and show the array based image using matplotlib module
plt.imshow(img_array)
plt.show()

In [ ]:
image_testing = Image.open('leaf_test.jpg')
image_testing = np.array(image_testing.resize((64,64))) / 255.0 # based on the image trained model size and normalize phase
image_testing.shape # show the testing image shape 

In [ ]:
image_testing = np.expand_dims(image_testing, axis=0)
print(image_testing.shape)

In [ ]:
output = model.predict(image_testing, 1)
print(output)
print(lb.classes_[output.argmax(Axis=1)])